In [1]:
from ngsolve import *
from xfem import *
from ngsolve.webgui import Draw
from os import mkdir
from copy import deepcopy
import dill as pickle
from util import *
from time import time
import matplotlib.pyplot as plt
from runOEDmethod import runOED

from functools import partial
from scipy.sparse.linalg import LinearOperator, cg
from scipy.linalg import solve

from tqdm.notebook import tqdm
from util import *
from safe_add_CFs import safe_add_CFs, interp_add_CFs

t1 = 1
delta_t = 1e-4

rng = np.random.default_rng()

robin = 1
alpha = 0.25
scale = 2

target_m = int(4e2)
maxh = 0.03
refine_inner = 0

clear = False

RDOED, RNGOED = runOED(
           target_m = target_m, integration_order = 5, 
           maxh = maxh, order = 2, refine_inner = refine_inner,
           tol = 1e-15,
           t1 = t1, delta_t = delta_t,
           robin = robin, alpha = alpha, scale = scale,
           sensor_radius = None, verbose = False,
           clear = clear, compute_all = False)

n, m, m_sensors, m_obs = RDOED.n, RDOED.m, RDOED.m_sensors, RDOED.m_obs

print("-"*40)
print("(n,m) =",(n,m))

ms = m_sensors
ones = np.ones(ms)
zero = np.zeros(ms)

mmaker = RDOED.mmaker
gmaker = RDOED.gmaker
gmaker.sensor_radius_square *= 1/2

rng = mmaker.rng

mesh = mmaker.mesh
fes = mmaker.fes
cofes = mmaker.cofes

print("m = ",m,", n = ",mmaker.n,", nco = ",cofes.ndof,", ell = ",RDOED.ell,sep="")

R = RDOED.R
Q = RDOED.Q
A = RDOED.A
AT = RDOED.AT
C = mmaker.C
O = gmaker.O
OT = gmaker.OT
F = RDOED.F
FT = RDOED.FT
#Fd = RDOED.Fd
#FdT = RDOED.FdT

real_to_ngs = mmaker.real_to_ngs
ngs_to_real = mmaker.ngs_to_real
coeff_to_ngs = mmaker.coeff_to_ngs
ngs_to_coeff = mmaker.ngs_to_coeff

mdigits = int(np.log10(m)+1)

fn = RDOED.output_filename

M = mmaker.M
dfes = mmaker.designfes
dmesh = mmaker.designmesh
    
eva = RDOED.J_init.eval
jac = RDOED.J_init.jac

order = 25
norm = lambda f: Integrate(f**2,mesh,order=order)**(1/2)

settings = {"Objects": {"Wireframe": False}, "Colormap" : { "ncolors" : 1024 }}

def withdesign(f,w,maxval=None):
    F = GridFunction(mmaker.designfes)
    F.Set(f)

    if maxval is None:
        maxval = ngs_to_max(F,F.space.mesh)

    FF = GridFunction(mmaker.codesignfes)
    FF.Set(F + gmaker.grid_to_ngs(w) * maxval / gmaker.peak)
    return FF

importing ngsxfem-2.1.2505


/home/caa/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


cor_length: 0.3 vs. domain radius 0.5 alpha: 0.25 , scale: 2
Loaded stored grid target_400_shape_pointwise_sensorradius_0_maxh_0.03_order_2_refine_0_dt_0.0001_robin_1_alpha_0.25_scale_2...
Red-Dom setup in 0:00:00.002671...
----------------------------------------
(n,m) = (5289, 418)
m = 418, n = 5289, nco = 17727, ell = 50


In [2]:
(eva(ones),eva(zero))

(np.float64(0.24256882271472885), np.float64(1.3469578673117262))

In [3]:
tests = 0
fes = mmaker.fes
cofes = mmaker.cofes

fes_to_fes = mmaker.fes_to_fes

Fd = RDOED.Fd
FdT = RDOED.FdT

Sghi = partial(RDOED.NoiseCovInverseHalf, include_noise_level = True)

def inner(f,g):
    
    if isinstance(f, np.ndarray) or isinstance(f, list):
        assert type(f) == type(g), "f is a vector, but g is not!"
        return np.vdot(g,f)
    return f.space.inner(f,g)

def norm(f):
    return inner(f,f)**(1/2)

def error(f,g):
    space = f.space
    err = GridFunction(space)
    err.vec.FV().NumPy()[:] = f.vec.FV().NumPy()[:] - g.vec.FV().NumPy()[:]
    return norm(err)

def relative_error(f,g):
    return error(f,g)/max(norm(f),norm(g))

err_fes_to_cofes = np.zeros(2)
err_cofes_to_fes = np.zeros(2)
err_prior = np.zeros(2)

err_PDE = np.zeros(2)
err_obs = np.zeros(2)
err_forward = np.zeros(2)
err_dforward = np.zeros(2)

err_model = np.zeros(2)
err_modeladj = np.zeros(2)

for i in tqdm(range(int(tests))):

    f = real_to_ngs(rng.normal(size=n))
    h = real_to_ngs(rng.normal(size=n))

    u = A(real_to_ngs(rng.normal(size=n)))
    v = A(real_to_ngs(rng.normal(size=n)))

    g = F(real_to_ngs(rng.normal(size=n)))
    d = F(real_to_ngs(rng.normal(size=n)))

    # FES-to-coFES adjoint inaccuracy
    inner1 = inner(f, fes_to_fes(v, fes))
    inner2 = inner(fes_to_fes(f, cofes), v)

    relerr = (np.abs(inner1-inner2)/max(np.abs(inner1),np.abs(inner2)))
    err_fes_to_cofes[0] += relerr/tests
    err_fes_to_cofes[1] = max(relerr,err_fes_to_cofes[1])
    
    # FES-to-coFES adjoint inaccuracy
    int1 = inner(u, fes_to_fes(h, cofes))
    int2 = inner(fes_to_fes(u, fes), h)

    relerr = (np.abs(inner1-inner2)/max(np.abs(inner1),np.abs(inner2)))
    err_cofes_to_fes[0] += relerr/tests
    err_cofes_to_fes[1] = max(relerr,err_cofes_to_fes[1])

    # Prior self-adjointness inaccuracy
    inner1 = inner(f,C(h))
    inner2 = inner(C(f),h)

    relerr = (np.abs(inner1-inner2)/max(np.abs(inner1),np.abs(inner2)))
    err_prior[0] += relerr/tests
    err_prior[1] = max(relerr,err_prior[1])

    # Adjoint PDE inaccuracy
    inner1 = inner(f,AT(v))
    inner2 = inner(A(f),v)

    relerr = (np.abs(inner1-inner2)/max(np.abs(inner1),np.abs(inner2)))
    err_PDE[0] += relerr/tests
    err_PDE[1] = max(relerr,err_PDE[1])

    # Obs adjoint inaccuracy
    inner1 = inner(u, OT(d))
    inner2 = inner(O(u), d)

    relerr = (np.abs(inner1-inner2)/max(np.abs(inner1),np.abs(inner2)))
    err_obs[0] += relerr/tests
    err_obs[1] = max(relerr,err_obs[1])

    # Forward adjoint inaccuracy
    inner1 = inner(f, FT(d))
    inner2 = inner(F(f), d)

    relerr = (np.abs(inner1-inner2)/max(np.abs(inner1),np.abs(inner2)))
    err_forward[0] += relerr/tests
    err_forward[1] = max(relerr,err_forward[1])

    # Discrete forward adjoint inaccuracy
    inner1 = inner(f, FdT(d))
    inner2 = inner(Fd(f), d)

    relerr = (np.abs(inner1-inner2)/max(np.abs(inner1),np.abs(inner2)))
    err_dforward[0] += relerr/tests
    err_dforward[1] = max(relerr,err_dforward[1])

    # Low-rank model error
    gd = Fd(f, include_noise = True)
    gf = Sghi(F(C(f)))
    
    relerr = np.linalg.norm(gf-gd)/max(np.linalg.norm(gf),np.linalg.norm(gd))
    err_model[0] += relerr/tests
    err_model[1] = max(relerr,err_model[1])

    # Low-rank model error
    fd = FdT(g, include_noise = True)
    fg = C(FT(Sghi(g)))
    
    relerr = relative_error(fd,fg)
    err_modeladj[0] += relerr/tests
    err_modeladj[1] = max(relerr,err_modeladj[1])
    
print("-"*40)
print("FES-COFES:               ",err_fes_to_cofes)
print("COFES-FES:               ",err_cofes_to_fes)

print("-"*40)
print("Prior:                   ",err_prior)
print("PDE adjoint:             ",err_PDE)
print("Obs adjoint:             ",err_obs)
print("Forward adjoint:         ",err_forward)
print("Discrete forward adjoint:",err_dforward)

print("-"*40)
print("Model error:             ",err_model)
print("Adjoint model error:     ",err_modeladj)

0it [00:00, ?it/s]

----------------------------------------
FES-COFES:                [0. 0.]
COFES-FES:                [0. 0.]
----------------------------------------
Prior:                    [0. 0.]
PDE adjoint:              [0. 0.]
Obs adjoint:              [0. 0.]
Forward adjoint:          [0. 0.]
Discrete forward adjoint: [0. 0.]
----------------------------------------
Model error:              [0. 0.]
Adjoint model error:      [0. 0.]


In [4]:
# Compute optimal designs

#targets = np.arange(1,m)
#max_target = 3#6
#targets = targets[targets <= max_target]

targets = np.arange(1,37)

try:
    with open(fn, "rb") as filename:
        obj = pickle.load(filename)
    print("Successfully loaded previous results.")
except:
    obj = {}
    
    obj["w1s"] = {}
    obj["ws"] = {}
    obj["wseqs"] = {}
    
    obj["ps"] = {}
    obj["times"] = {}
    
w1 = None

for target in targets:
    
    # Try to load optimal design from file
    if target in obj["ws"].keys():
        print("Skipping ",target,"...",sep="")
    else:
        start = time()
        RDOED.Opt(target = target, w1 = w1, verbose = False)
        
        #w0 = deepcopy(RDOED.design)
        w1 = deepcopy(RDOED.global_design)
        
        obj["w1s"][target] = deepcopy(w1)
        obj["ws"][target] = deepcopy(RDOED.design)
        obj["wseqs"][target] = deepcopy(RDOED.design_sequence)
        try:
            obj["ps"][target] = deepcopy(RDOED.ps)
        except:
            obj["ps"][target] = None
        obj["times"][target] = time() - start
    
        with open(fn, "wb") as filename:
            pickle.dump(obj, filename)

Successfully loaded previous results.
Skipping 1...
Skipping 2...
Skipping 3...
Skipping 4...
Skipping 5...
Skipping 6...
Skipping 7...
Skipping 8...
Skipping 9...
Skipping 10...
Skipping 11...
Skipping 12...
Skipping 13...
Skipping 14...
Skipping 15...
Skipping 16...
Skipping 17...
Skipping 18...
Skipping 19...
Skipping 20...
Skipping 21...
Skipping 22...
Skipping 23...
Skipping 24...
Skipping 25...
Skipping 26...
Skipping 27...
Skipping 28...
Skipping 29...
Skipping 30...
Skipping 31...
Skipping 32...
Skipping 33...
Skipping 34...
Skipping 35...
Skipping 36...


In [5]:
fnRNG = fn + "_RNG"
try:
    with open(fnRNG, "rb") as filename:
        objRNG = pickle.load(filename)
    print("Successfully loaded previous results.")
except:
    objRNG = {}
    
    objRNG["ws"] = {}
    objRNG["allvals"] = {}
    objRNG["times"] = {}

for target in targets:
    
    # Try to load optimal design from file
    if target in objRNG["ws"].keys():
        print("Skipping ",target,"...",sep="")
    else:
        print("RNG for target ",target,"...",sep="")
        start = time()
        RNGOED.RNG(target = target, tries = int(1e3), verbose = False)
        
        wRNG = deepcopy(RNGOED.design)
        allvals = deepcopy(RNGOED.allvals)
        
        objRNG["ws"][target] = deepcopy(wRNG)
        objRNG["allvals"][target] = deepcopy(allvals)
        objRNG["times"][target] = time() - start
    
        with open(fnRNG, "wb") as filename:
            pickle.dump(objRNG, filename)

Successfully loaded previous results.
Skipping 1...
Skipping 2...
Skipping 3...
Skipping 4...
Skipping 5...
Skipping 6...
Skipping 7...
Skipping 8...
Skipping 9...
Skipping 10...
Skipping 11...
Skipping 12...
Skipping 13...
Skipping 14...
Skipping 15...
Skipping 16...
Skipping 17...
Skipping 18...
Skipping 19...
Skipping 20...
Skipping 21...
Skipping 22...
Skipping 23...
Skipping 24...
Skipping 25...
Skipping 26...
Skipping 27...
Skipping 28...
Skipping 29...
Skipping 30...
Skipping 31...
Skipping 32...
Skipping 33...
Skipping 34...
Skipping 35...
Skipping 36...


In [ ]:
order = 25
target = 36#targets[-1]#max_target

if 1:
    w = obj["ws"][target]
    w1 = obj["w1s"][target]
    wRNG = objRNG["ws"][target]
else:
    w = ones
    w1 = ones
    wRNG = ones
    
f = GridFunction(fes)
#f.Set(7 * cos(5*y)**2)
if 0:
    f.Set(exp(-40*((x+0.9)**2+y**2)))
    f.vec.data /= Integrate(f*f,mesh)**(1/2)
    f = C(f)
    f = AT(A(f, t1 = t1), t1 = t1)
elif 0:
    f.Set(1)
elif 0:
    box_S = 0.5
    box_W = -0.9
    box_E = -0.7
    box_N = 0.7
    box = IfPos(y - box_S,IfPos(box_N - y, IfPos(x - box_W, IfPos(box_E - x, 1, 0), 0), 0), 0)
    cf = box#f.Set(box)
elif 0:
    r = 40
    cf = exp(-r*((x+0.75)**2+(y+0.5)**2))
    cf += exp(-r*((x+0.75)**2+(y-0.75)**2))
else:
    r = 3/40 * np.pi
    power = 1/8
    
    x1 = -0.75
    y1 = -0.7

    x2 = -0.75
    y2 = 0.7

    r1 = r**2 - (x-x1)**2 - (y-y1)**2
    r2 = r**2 - (x-x2)**2 - (y-y2)**2

    cf1 = IfPos(r1,exp(-1/r1**power),0)
    cf2 = IfPos(r2,exp(-1/r2**power),0)
    cf = cf1+cf2
    #f.Set(cf1+cf2)

f = GridFunction(fes)
if 1:
    u,v = fes.TnT()
    vdual = v.Operator("dual")
    
    a = BilinearForm(fes)
    a += u*vdual*dx + u*vdual*dx(element_vb=BND) + \
        u*vdual*dx(element_vb=BBND)
    a.Assemble()
    
    Lf = LinearForm(fes)
    Lf += cf*vdual*dx + cf*vdual*dx(element_vb=BND) + \
        cf*vdual*dx(element_vb=BBND)
    Lf.Assemble()
    
    # interpolation in vertices preserves values 0 and 1
    f.vec.data = a.mat.Inverse() * Lf.vec
else:
    #pass
    f.Set(cf)

def fb(r, f = None, w = None, alpha = 0):
    if f is None:
        f = mmaker.real_to_ngs(r)
    f = C(f)
    f = F(f, obs_times = [t1])
    if w is not None:
        f = RDOED.W(w) * f
    f = FT(f, obs_times = [t1])
    f = C(f)
    return f#mmaker.ngs_to_real(f)

if 0:
    
    
    def callback(sol_vec):
        relative_residual_norm = np.linalg.norm(r - fb(sol_vec)) / np.linalg.norm(r)
        print("\f",relative_residual_norm)
    
    r = mmaker.ngs_to_real(f)
    r, _ = cg(A = LinearOperator(matvec = lambda r: fb(r, alpha = 1e-5), shape = (n,n)), b = fb(r), x0 = np.zeros(n), rtol = 1e-15, callback = callback)
    f = mmaker.real_to_ngs(r)
    
    Draw(f)

#f = FT(F(f, obs_times = [t1]), obs_times = [t1])
#f = C(cf)
#f = C(C(cf))
#f = fb(r = None, f = f, w = w)
#f.Set(cf)
#f = mmaker.real_to_ngs(f)
f.vec.data /= ngs_to_max(f,mesh)#norm(mmaker.fieldPrior) / norm(f)

In [7]:
if 0:
    fmax = ngs_to_max(f,mesh)
    f.vec.data /= fmax
    f.vec.data *= 1e3
    fmax = 1e3

    Draw(f, dmesh, min = 0, max = fmax, settings=settings)

    u = A(f,t1=t1)

    Draw(u, dmesh, min = 0, settings=settings)

    g = O(u)
    gdagger = 1*g

    fac = 1

    plt.plot(g,'b')
    g += RDOED.NoiseCovHalf(rng.normal(size=m), include_noise_level=True)
    #plt.plot(g,'r')
    plt.show()

    plt.plot(gdagger-g,'r')
    plt.show()

    normf = norm(f)
    normu = norm(u)
    normg = np.linalg.norm(g)

    print("Norm f: ",normf,", norm u: ",normu,", norm g: ",normg,"...",sep="")

    fmax = ngs_to_max(f,mesh)
    umax = ngs_to_max(u,mesh)

    w = obj["ws"][36]
    wRNG = objRNG["ws"][36]

    frs = []
    urs = []
    grs = []

    for W in [w, wRNG]:

        fr = RDOED.design_to_sol(w = W, g = g, fac = fac)
        ur = A(fr, t1 = t1)
        gr = O(ur)

        frs.append(fr)
        urs.append(ur)
        grs.append(gr)

        #mymax = ngs_to_max(fr,mesh)
        #if np.abs(mymax-fmax)/fmax < 0.3:
        #    mymax = fmax
        #else:
        #    mymax = min(mymax,fmax)
        
        Draw(withdesign(fr,W),dmesh, min = 0, max = fmax, settings=settings)
        Draw(ur,mesh, min = 0, settings=settings)
        err = relative_error(f,fr)
        res = np.linalg.norm(gr-g)/np.linalg.norm(g)
        
        print("Relative reco error      : ",err,", residual: ",res,"...",sep="")

        continue

        fre = RDOED.design_to_sol_exact(w = W, g = g, fac = fac)
        gre = F(fre)

        #mymax = ngs_to_max(fre,mesh)
        #if np.abs(mymax-fmax)/fmax < 0.3:
        #    mymax = fmax
        #else:
        #    mymax = min(mymax,fmax)
        
        Draw(withdesign(fre,W),dmesh, min = 0, max = fmax, settings=settings)
        erre = relative_error(f,fre)
        rese = np.linalg.norm(gre-g)/np.linalg.norm(g)

        print("Relative exact reco error: ",erre,", residual: ",rese,"...",sep="")
        
        print("-"*40)

    f.Save("f")
    u.Save("u")

    frs[0].Save("fr_OED")
    urs[0].Save("ur_OED")

    frs[1].Save("fr_RNG")
    urs[1].Save("ur_RNG")

    1-relative_error(f,frs[0])/relative_error(f,frs[1])

In [8]:
field = RDOED.design_to_field(w = w, interp = True)
field.Save("field")
Draw(withdesign(field, w = w), mesh, settings=settings)

fieldRNG = RDOED.design_to_field(w = wRNG, interp = True)
Draw(withdesign(fieldRNG, w = wRNG), mesh, settings=settings)

print(eva(w))
print(Integrate(field,mesh))
print(eva(wRNG))
print(Integrate(fieldRNG,mesh))

Biggest eigval: 1.3347326984460314e-19, smallest: -0.6429771461316857, ratio: -4.817272753415532e+18...


KeyboardInterrupt: 